## Fasion MNIST classification and hyperparameter optimization using Scikit-Optmizer

Here we will be concerned with optimizing hyperparameters for a Convolutional Neural Network to get better results in the task of recognizing images from a data set. We will try to find the best ( or at least good) values for 3 parameters, The learning rate for the Adam optmizer, the numer of dense layers and the number of neurons in each dense layer. To find the best values for the 3 parameters, we have some options, we could try all combinations of parameters, wich is an option, but not a good one, if we were to try 10 different values for each parameter, eu would have to make 1000 model evaluations and it could take too much time, this is called grid search. We could also try random values for some time and then save the best set, and well, it does not sound promising, does it?

First let's talk a little bit about Bayesian optimization using Gaussian Processes.

Gaussian processes provide not only the value for some function, but also the uncertainty around that value. Bayesian Optimization is a class of iterative optimization methods that focuses on the general optimization setting, where a set of values o hyperparameters is known (the vector space) but there is no information about the function we are trying to optimize. As the process is experimented, the optimizer gains more confidence in what the performace function looks like and what values can optimize it.


![1_UWvMP_dtt1lQhwQvfNOjLw](https://user-images.githubusercontent.com/23335136/55368076-5ce0b980-54c6-11e9-88e8-a17f6ab7a20f.png)

The performance is a function of the hyperparameters, and is not known at the beggining, but as the experiments progress, the optimizer gets more and more confident in what that function looks like, this update in believe is made using Bayes Rule, the function that will get the next hyperparameters to experiment, acquisition function, is called Expected Improvement, and as the name sugests, it chooses the hyperparameters it expects will get the best improvements balancing exploration and exploitation.

A better explanation (also longer) can be found here: https://www.youtube.com/watch?v=jtRPxRnOXnk&t=2018s


An even better explanation with way better code (but code is longer) https://github.com/Hvass-Labs/TensorFlow-Tutorials/blob/master/19_Hyper-Parameters.ipynb


In [1]:
# Import TensorFlow and TensorFlow Datasets
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import backend as K
tf.logging.set_verbosity(tf.logging.ERROR)

# Helper libraries
import math
import numpy as np
import matplotlib.pyplot as plt

import skopt
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.plots import plot_convergence
from skopt.plots import plot_objective, plot_evaluations
from skopt.utils import use_named_args


print("tensorflow GPU Version ",tf.__version__)

tensorflow GPU Version  1.12.0


We will use de fashion MNIST in this problem, it has 70000 images and 10 classes like the good old MNIST, but has more complexity in its shapes and is a better challange. Some samples from this dataset can be seen in the image bellow

<table>
  <tr><td>
    <img src="https://tensorflow.org/images/fashion-mnist-sprite.png"
         alt="Fashion MNIST sprite"  width="600">
  </td></tr>
  <tr><td align="center">
    <b> </b> <a href="https://github.com/zalandoresearch/fashion-mnist">Fashion-MNIST samples</a> (by Zalando, MIT License).<br/>&nbsp;
  </td></tr>
</table>


We will download the dataset set using tensorflow datasets, if you dont have it, $pip install tensorflow_datasets

In [2]:
dataset, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

Then we take 60000 exemples for training and 10000 for testing, as the dataset is somewhat little, we will not have an evaluating set. We will do data exploration here, the notebook is already long enought

In [3]:
num_train_examples = metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples
print("Number of training examples: {}".format(num_train_examples))
print("Number of test examples:     {}".format(num_test_examples))

Number of training examples: 60000
Number of test examples:     10000


We need to normalize the images, changing the pixel intensity to range from 0-1 will speed up convergence while training. Each pixel intensity has 8 bits in RGB images, so we need to divide each image by 2^8-1=255 as the images in the dataset have only one channel

In [4]:
def normalize(images, labels):
    
    """
    Normalizes images pixel intensity to range from 0-1

    """
    images = tf.cast(images, tf.float32)

    images /= 255
    return images, labels

# The map function applies the normalize function to each element in the train
# and test datasets
train_dataset =  train_dataset.map(normalize)
test_dataset  =  test_dataset.map(normalize)

Next we need to create a function to create models with the different combinations of hyperparameters for the gaussian process optimizer, this function will be called in every iteration of the optimizer and needs to return a compiled model to be trained and evaluated

In [5]:
def create_model(learning_rate, num_dense_layers,
                 num_dense_nodes):
    """
    Hyper-parameters:
    learning_rate:     Learning-rate for the optimizer.
    num_dense_layers:  Number of dense layers.
    num_dense_nodes:   Number of nodes in each dense layer.
    
    Returns: compiled model
    """
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu,
                           input_shape=(28, 28, 1)))
    model.add(tf.keras.layers.MaxPooling2D((2, 2), strides=2))
    model.add(tf.keras.layers.Conv2D(64, (3,3), padding='same', activation=tf.nn.relu))
    model.add(tf.keras.layers.MaxPooling2D((2, 2), strides=2))
    model.add(tf.keras.layers.Flatten())
        
    for i in range(num_dense_layers):
        model.add(tf.keras.layers.Dense(num_dense_nodes, activation=tf.nn.relu))
        
    model.add(tf.keras.layers.Dense(10,  activation=tf.nn.softmax))
        
    optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
        
    model.compile(optimizer=optimizer, 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

    return model

Let's run one model to see how it goes

In [6]:
#create model with lr = 0.001, 1 dense layer with 12 nodes
model = create_model(0.001, 1, 12)

BATCH_SIZE = 512
#if you get crashes during training, try lowering BATCH_SIZE

train_dataset = train_dataset.repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

The training phase will use the 60000 training images 3 times,  ceil(60000/512) = 118 iterations each epoch

In [7]:
model.fit(train_dataset, epochs=3, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))

Epoch 1/3
118/118 [==============================] - ETA: 17:12 - loss: 2.3015 - acc: 0.13 - ETA: 8:39 - loss: 2.2927 - acc: 0.1240 - ETA: 5:47 - loss: 2.2795 - acc: 0.125 - ETA: 4:22 - loss: 2.2644 - acc: 0.134 - ETA: 3:30 - loss: 2.2500 - acc: 0.134 - ETA: 2:55 - loss: 2.2330 - acc: 0.141 - ETA: 2:30 - loss: 2.2164 - acc: 0.146 - ETA: 2:11 - loss: 2.1999 - acc: 0.152 - ETA: 1:56 - loss: 2.1846 - acc: 0.159 - ETA: 1:44 - loss: 2.1685 - acc: 0.164 - ETA: 1:35 - loss: 2.1544 - acc: 0.167 - ETA: 1:26 - loss: 2.1393 - acc: 0.174 - ETA: 1:20 - loss: 2.1265 - acc: 0.177 - ETA: 1:14 - loss: 2.1068 - acc: 0.183 - ETA: 1:09 - loss: 2.0930 - acc: 0.184 - ETA: 1:04 - loss: 2.0753 - acc: 0.189 - ETA: 1:00 - loss: 2.0592 - acc: 0.193 - ETA: 57s - loss: 2.0434 - acc: 0.200 - ETA: 54s - loss: 2.0254 - acc: 0.20 - ETA: 51s - loss: 2.0086 - acc: 0.21 - ETA: 48s - loss: 1.9914 - acc: 0.22 - ETA: 46s - loss: 1.9737 - acc: 0.23 - ETA: 44s - loss: 1.9562 - acc: 0.23 - ETA: 42s - loss: 1.9419 - acc: 0.24 -

118/118 [==============================] - ETA: 13s - loss: 0.7524 - acc: 0.72 - ETA: 13s - loss: 0.7621 - acc: 0.72 - ETA: 13s - loss: 0.7478 - acc: 0.73 - ETA: 13s - loss: 0.7475 - acc: 0.72 - ETA: 12s - loss: 0.7739 - acc: 0.71 - ETA: 11s - loss: 0.7807 - acc: 0.71 - ETA: 11s - loss: 0.7777 - acc: 0.71 - ETA: 10s - loss: 0.7761 - acc: 0.71 - ETA: 10s - loss: 0.7717 - acc: 0.71 - ETA: 10s - loss: 0.7728 - acc: 0.71 - ETA: 9s - loss: 0.7691 - acc: 0.7145 - ETA: 9s - loss: 0.7697 - acc: 0.716 - ETA: 9s - loss: 0.7649 - acc: 0.717 - ETA: 9s - loss: 0.7558 - acc: 0.721 - ETA: 9s - loss: 0.7592 - acc: 0.721 - ETA: 9s - loss: 0.7612 - acc: 0.720 - ETA: 8s - loss: 0.7584 - acc: 0.721 - ETA: 8s - loss: 0.7572 - acc: 0.720 - ETA: 8s - loss: 0.7574 - acc: 0.720 - ETA: 8s - loss: 0.7540 - acc: 0.722 - ETA: 8s - loss: 0.7525 - acc: 0.723 - ETA: 8s - loss: 0.7477 - acc: 0.725 - ETA: 8s - loss: 0.7442 - acc: 0.727 - ETA: 8s - loss: 0.7410 - acc: 0.727 - ETA: 7s - loss: 0.7364 - acc: 0.729 - ETA: 7

118/118 [==============================] - ETA: 13s - loss: 0.5688 - acc: 0.79 - ETA: 12s - loss: 0.4824 - acc: 0.82 - ETA: 13s - loss: 0.4706 - acc: 0.82 - ETA: 12s - loss: 0.4725 - acc: 0.83 - ETA: 12s - loss: 0.4697 - acc: 0.83 - ETA: 11s - loss: 0.4671 - acc: 0.83 - ETA: 11s - loss: 0.4523 - acc: 0.84 - ETA: 10s - loss: 0.4478 - acc: 0.84 - ETA: 10s - loss: 0.4484 - acc: 0.84 - ETA: 10s - loss: 0.4439 - acc: 0.84 - ETA: 9s - loss: 0.4433 - acc: 0.8429 - ETA: 9s - loss: 0.4410 - acc: 0.844 - ETA: 9s - loss: 0.4387 - acc: 0.846 - ETA: 9s - loss: 0.4393 - acc: 0.845 - ETA: 9s - loss: 0.4354 - acc: 0.847 - ETA: 8s - loss: 0.4357 - acc: 0.848 - ETA: 8s - loss: 0.4406 - acc: 0.846 - ETA: 8s - loss: 0.4430 - acc: 0.845 - ETA: 8s - loss: 0.4399 - acc: 0.846 - ETA: 8s - loss: 0.4385 - acc: 0.846 - ETA: 8s - loss: 0.4340 - acc: 0.847 - ETA: 8s - loss: 0.4342 - acc: 0.847 - ETA: 8s - loss: 0.4333 - acc: 0.847 - ETA: 8s - loss: 0.4346 - acc: 0.847 - ETA: 7s - loss: 0.4368 - acc: 0.845 - ETA: 7

Lets see how the model does on the test set

In [8]:
test_loss, test_accuracy = model.evaluate(test_dataset, steps=math.ceil(num_test_examples/BATCH_SIZE))

20/20 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 1s 74ms/step


We got a fairly good model, let's see if we can find one better. Now to the optmizations part

In [9]:
print("Acc : ",test_accuracy)

# Delete the Keras model with these hyper-parameters from memory so we get create a new one
# in the optimization part
del model
K.clear_session()
    

Acc :  0.85341796875


First we need to create a list with hyperparamters and the range they are allowed

In [10]:
# the learning rate will be updated with logritmic scale, beucase it has a wide range
dim_learning_rate = Real(low=1e-5, high=1e-1, prior='log-uniform',name='learning_rate')
dim_num_dense_layers = Integer(low=1, high=5, name='num_dense_layers')
dim_num_dense_nodes = Integer(low=5, high=512, name='num_dense_nodes')

dimensions = [dim_learning_rate,
              dim_num_dense_layers,
              dim_num_dense_nodes]

Now we create the function that will evaluate our model and return its performance to the optimizer

In [11]:
@use_named_args(dimensions=dimensions)
def fitness(learning_rate, num_dense_layers,
            num_dense_nodes):
    """
    Hyper-parameters:
    learning_rate:     Learning-rate for the optimizer.
    num_dense_layers:  Number of dense layers.
    num_dense_nodes:   Number of nodes in each dense layer.
    """
    
    # we will be updating best_accuracy and best_params as the optimizer call this function
    # so the two will be global variables
    global best_accuracy
    global best_params
    
    # Create the neural network with these hyper-parameters.
    model = create_model(learning_rate=learning_rate,
                         num_dense_layers=num_dense_layers,
                         num_dense_nodes=num_dense_nodes)
    
    # for some reason, we have to load the dataset all over again, i suppose it is because
    # we close and clear the session, and the dataset is loaded by tensorflow
    BATCH_SIZE = 512
    dataset, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True)
    train_dataset, test_dataset = dataset['train'], dataset['test']
    train_dataset =  train_dataset.map(normalize)
    test_dataset  =  test_dataset.map(normalize)
    train_dataset = train_dataset.repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
    test_dataset = test_dataset.batch(BATCH_SIZE)

    # Train network
    model.fit(train_dataset, epochs=3, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))
    
    #evaluate it
    test_loss, test_accuracy = model.evaluate(test_dataset, steps=math.ceil(num_test_examples/BATCH_SIZE))

    # if it did better than the last one, we sabe parameters and acc
    if test_accuracy > best_accuracy:

        best_params = [learning_rate, num_dense_layers,num_dense_nodes]
        best_accuracy = test_accuracy

    # Delete the Keras model with these hyper-parameters from memory so we get create a new one
    # in the next iteration
    del model
    K.clear_session()
    
    # NOTE: Scikit-optimize does minimization so it tries to
    # find a set of hyper-parameters with the LOWEST fitness-value.
    # Because we are interested in the HIGHEST classification
    # accuracy, we need to negate this number so it can be minimized.
    return -test_accuracy

Everything is ready, now we call the our gaussian process optimizer

In [12]:
# We need to give the optimizer a starting point, so it will start with the
# hyperparameters with tryed
default_parameters = [0.001,1,12]

# We will say that the last model has the best hyperparams and acc
# and if the optimizer finds better ones, they will be updated
best_accuracy = test_accuracy
best_params = default_parameters

# gp_minimize is our glorious gaussian process optimizer
# n_call default is 100 it cannot be low or the optimizer wont work
search_result = gp_minimize(func=fitness,
                            dimensions=dimensions,
                            acq_func='EI', # Expected Improvement.
                            n_calls=60,
                            x0=default_parameters)


Epoch 1/3
118/118 [==============================] - ETA: 15:21 - loss: 2.3030 - acc: 0.15 - ETA: 7:44 - loss: 2.2908 - acc: 0.1455 - ETA: 5:12 - loss: 2.2799 - acc: 0.156 - ETA: 3:55 - loss: 2.2712 - acc: 0.158 - ETA: 3:09 - loss: 2.2575 - acc: 0.177 - ETA: 2:38 - loss: 2.2441 - acc: 0.191 - ETA: 2:15 - loss: 2.2264 - acc: 0.207 - ETA: 1:58 - loss: 2.2091 - acc: 0.217 - ETA: 1:45 - loss: 2.1923 - acc: 0.226 - ETA: 1:35 - loss: 2.1780 - acc: 0.230 - ETA: 1:26 - loss: 2.1588 - acc: 0.238 - ETA: 1:19 - loss: 2.1424 - acc: 0.244 - ETA: 1:13 - loss: 2.1258 - acc: 0.249 - ETA: 1:07 - loss: 2.1082 - acc: 0.252 - ETA: 1:03 - loss: 2.0892 - acc: 0.256 - ETA: 59s - loss: 2.0700 - acc: 0.260 - ETA: 55s - loss: 2.0524 - acc: 0.26 - ETA: 52s - loss: 2.0322 - acc: 0.26 - ETA: 49s - loss: 2.0159 - acc: 0.27 - ETA: 46s - loss: 1.9984 - acc: 0.27 - ETA: 44s - loss: 1.9801 - acc: 0.28 - ETA: 42s - loss: 1.9629 - acc: 0.28 - ETA: 40s - loss: 1.9465 - acc: 0.28 - ETA: 38s - loss: 1.9282 - acc: 0.29 - ETA

118/118 [==============================] - ETA: 13s - loss: 0.5661 - acc: 0.81 - ETA: 13s - loss: 0.6095 - acc: 0.80 - ETA: 13s - loss: 0.5678 - acc: 0.81 - ETA: 12s - loss: 0.5771 - acc: 0.81 - ETA: 11s - loss: 0.5678 - acc: 0.82 - ETA: 11s - loss: 0.5680 - acc: 0.81 - ETA: 10s - loss: 0.5601 - acc: 0.81 - ETA: 10s - loss: 0.5546 - acc: 0.81 - ETA: 9s - loss: 0.5557 - acc: 0.8151 - ETA: 9s - loss: 0.5545 - acc: 0.816 - ETA: 9s - loss: 0.5590 - acc: 0.814 - ETA: 9s - loss: 0.5596 - acc: 0.814 - ETA: 9s - loss: 0.5569 - acc: 0.814 - ETA: 9s - loss: 0.5602 - acc: 0.811 - ETA: 8s - loss: 0.5604 - acc: 0.810 - ETA: 8s - loss: 0.5609 - acc: 0.809 - ETA: 8s - loss: 0.5556 - acc: 0.812 - ETA: 8s - loss: 0.5564 - acc: 0.811 - ETA: 8s - loss: 0.5532 - acc: 0.812 - ETA: 8s - loss: 0.5532 - acc: 0.813 - ETA: 8s - loss: 0.5527 - acc: 0.812 - ETA: 8s - loss: 0.5500 - acc: 0.813 - ETA: 7s - loss: 0.5463 - acc: 0.815 - ETA: 7s - loss: 0.5471 - acc: 0.814 - ETA: 7s - loss: 0.5445 - acc: 0.815 - ETA: 7

20/20 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 2s 75ms/step
Epoch 1/3


118/118 [==============================] - ETA: 15:29 - loss: 2.3102 - acc: 0.08 - ETA: 7:48 - loss: 4.3633 - acc: 0.1992 - ETA: 5:14 - loss: 4.7609 - acc: 0.165 - ETA: 3:57 - loss: 4.1535 - acc: 0.147 - ETA: 3:10 - loss: 3.7855 - acc: 0.135 - ETA: 2:38 - loss: 3.9062 - acc: 0.125 - ETA: 2:16 - loss: 3.6777 - acc: 0.122 - ETA: 1:59 - loss: 3.5060 - acc: 0.121 - ETA: 1:46 - loss: 3.3730 - acc: 0.119 - ETA: 1:35 - loss: 3.2666 - acc: 0.115 - ETA: 1:26 - loss: 3.1796 - acc: 0.114 - ETA: 1:19 - loss: 3.1069 - acc: 0.113 - ETA: 1:13 - loss: 3.0452 - acc: 0.113 - ETA: 1:08 - loss: 2.9924 - acc: 0.112 - ETA: 1:03 - loss: 2.9466 - acc: 0.111 - ETA: 59s - loss: 2.9066 - acc: 0.110 - ETA: 56s - loss: 2.8714 - acc: 0.10 - ETA: 53s - loss: 2.8400 - acc: 0.10 - ETA: 50s - loss: 2.8116 - acc: 0.10 - ETA: 47s - loss: 2.7860 - acc: 0.11 - ETA: 45s - loss: 2.7635 - acc: 0.10 - ETA: 43s - loss: 2.7427 - acc: 0.10 - ETA: 41s - loss: 2.7237 - acc: 0.10 - ETA: 39s - loss: 2.7061 - acc: 0.10 - ETA: 37s - lo

118/118 [==============================] - ETA: 14s - loss: 2.3035 - acc: 0.06 - ETA: 14s - loss: 2.3026 - acc: 0.09 - ETA: 14s - loss: 2.3026 - acc: 0.09 - ETA: 14s - loss: 2.3026 - acc: 0.09 - ETA: 13s - loss: 2.3026 - acc: 0.10 - ETA: 12s - loss: 2.3028 - acc: 0.10 - ETA: 12s - loss: 2.3028 - acc: 0.10 - ETA: 11s - loss: 2.3028 - acc: 0.10 - ETA: 11s - loss: 2.3029 - acc: 0.10 - ETA: 11s - loss: 2.3029 - acc: 0.09 - ETA: 10s - loss: 2.3028 - acc: 0.09 - ETA: 10s - loss: 2.3030 - acc: 0.09 - ETA: 10s - loss: 2.3030 - acc: 0.09 - ETA: 10s - loss: 2.3029 - acc: 0.09 - ETA: 9s - loss: 2.3028 - acc: 0.0986 - ETA: 9s - loss: 2.3029 - acc: 0.098 - ETA: 9s - loss: 2.3029 - acc: 0.098 - ETA: 9s - loss: 2.3030 - acc: 0.099 - ETA: 9s - loss: 2.3029 - acc: 0.099 - ETA: 9s - loss: 2.3031 - acc: 0.099 - ETA: 9s - loss: 2.3030 - acc: 0.100 - ETA: 8s - loss: 2.3033 - acc: 0.099 - ETA: 8s - loss: 2.3033 - acc: 0.099 - ETA: 8s - loss: 2.3032 - acc: 0.099 - ETA: 8s - loss: 2.3033 - acc: 0.098 - ETA: 8

...

118/118 [==============================] - ETA: 16s - loss: 0.5028 - acc: 0.83 - ETA: 16s - loss: 0.5190 - acc: 0.82 - ETA: 15s - loss: 0.5150 - acc: 0.82 - ETA: 14s - loss: 0.5214 - acc: 0.81 - ETA: 13s - loss: 0.5194 - acc: 0.81 - ETA: 12s - loss: 0.5207 - acc: 0.81 - ETA: 12s - loss: 0.5156 - acc: 0.81 - ETA: 11s - loss: 0.5234 - acc: 0.80 - ETA: 11s - loss: 0.5177 - acc: 0.81 - ETA: 11s - loss: 0.5173 - acc: 0.81 - ETA: 10s - loss: 0.5230 - acc: 0.81 - ETA: 10s - loss: 0.5185 - acc: 0.81 - ETA: 10s - loss: 0.5196 - acc: 0.81 - ETA: 10s - loss: 0.5204 - acc: 0.81 - ETA: 10s - loss: 0.5209 - acc: 0.81 - ETA: 10s - loss: 0.5231 - acc: 0.81 - ETA: 10s - loss: 0.5247 - acc: 0.81 - ETA: 10s - loss: 0.5238 - acc: 0.81 - ETA: 9s - loss: 0.5235 - acc: 0.8112 - ETA: 9s - loss: 0.5248 - acc: 0.810 - ETA: 9s - loss: 0.5255 - acc: 0.811 - ETA: 9s - loss: 0.5222 - acc: 0.812 - ETA: 9s - loss: 0.5223 - acc: 0.812 - ETA: 9s - loss: 0.5239 - acc: 0.812 - ETA: 9s - loss: 0.5219 - acc: 0.813 - ETA: 9

20/20 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 2s 90ms/step
Epoch 1/3


118/118 [==============================] - ETA: 18:59 - loss: 2.3156 - acc: 0.06 - ETA: 9:34 - loss: 2.3048 - acc: 0.1055 - ETA: 6:26 - loss: 2.2963 - acc: 0.145 - ETA: 4:51 - loss: 2.2865 - acc: 0.183 - ETA: 3:54 - loss: 2.2770 - acc: 0.215 - ETA: 3:15 - loss: 2.2677 - acc: 0.238 - ETA: 2:47 - loss: 2.2584 - acc: 0.251 - ETA: 2:26 - loss: 2.2496 - acc: 0.271 - ETA: 2:10 - loss: 2.2401 - acc: 0.294 - ETA: 1:57 - loss: 2.2309 - acc: 0.310 - ETA: 1:46 - loss: 2.2213 - acc: 0.331 - ETA: 1:37 - loss: 2.2132 - acc: 0.343 - ETA: 1:29 - loss: 2.2028 - acc: 0.361 - ETA: 1:23 - loss: 2.1925 - acc: 0.373 - ETA: 1:17 - loss: 2.1816 - acc: 0.388 - ETA: 1:12 - loss: 2.1706 - acc: 0.402 - ETA: 1:08 - loss: 2.1600 - acc: 0.412 - ETA: 1:04 - loss: 2.1485 - acc: 0.422 - ETA: 1:00 - loss: 2.1367 - acc: 0.433 - ETA: 57s - loss: 2.1246 - acc: 0.443 - ETA: 54s - loss: 2.1126 - acc: 0.45 - ETA: 52s - loss: 2.1002 - acc: 0.45 - ETA: 49s - loss: 2.0868 - acc: 0.47 - ETA: 47s - loss: 2.0732 - acc: 0.47 - ETA: 

118/118 [==============================] - ETA: 17s - loss: 0.6288 - acc: 0.75 - ETA: 16s - loss: 0.6258 - acc: 0.77 - ETA: 17s - loss: 0.6066 - acc: 0.78 - ETA: 16s - loss: 0.6125 - acc: 0.78 - ETA: 15s - loss: 0.6159 - acc: 0.77 - ETA: 14s - loss: 0.6148 - acc: 0.78 - ETA: 13s - loss: 0.6178 - acc: 0.77 - ETA: 13s - loss: 0.6179 - acc: 0.77 - ETA: 12s - loss: 0.6136 - acc: 0.77 - ETA: 12s - loss: 0.6151 - acc: 0.77 - ETA: 11s - loss: 0.6081 - acc: 0.78 - ETA: 11s - loss: 0.6041 - acc: 0.78 - ETA: 11s - loss: 0.6031 - acc: 0.78 - ETA: 11s - loss: 0.6000 - acc: 0.78 - ETA: 11s - loss: 0.5952 - acc: 0.78 - ETA: 10s - loss: 0.5938 - acc: 0.78 - ETA: 10s - loss: 0.5939 - acc: 0.78 - ETA: 10s - loss: 0.5908 - acc: 0.78 - ETA: 10s - loss: 0.5881 - acc: 0.79 - ETA: 10s - loss: 0.5893 - acc: 0.79 - ETA: 10s - loss: 0.5868 - acc: 0.79 - ETA: 9s - loss: 0.5835 - acc: 0.7932 - ETA: 9s - loss: 0.5813 - acc: 0.794 - ETA: 9s - loss: 0.5817 - acc: 0.793 - ETA: 9s - loss: 0.5833 - acc: 0.792 - ETA: 9

20/20 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 2s 85ms/step


In [13]:
print("For this data set, the best hyperparameters found are :")
print("Learn Rate : ",best_params[0])
print("Num of dense layers : ",best_params[1])
print("Num of dense nodes : ",best_params[2])
print("It got an accuracy of :",best_accuracy)

For this data set, the best hyperparameters found are :
Learn Rate :  0.007747151042603749
Num of dense layers :  1
Num of dense nodes :  296
It got an accuracy of : 0.9097886025905609


## Conclusion

Tuning hyperparameters can be hard and time consuming, and it can get even harder if you don't have past experiences to get the feeling of how a CNN should look like to be good at classification tasks.
Here the gp optimizer took 60 iterations and could find a better set of hyperparameter than my first guess, it confirmed that one dense layer is the best choice, maybe more layers would overfit easily, but it find 296 nodes a better option, wich is more than 20 times my first guess, the learn rate was not so far of my first guess.
There is a good chance the optimizer would find an even better set of hyperparameters as it was still exploring the space of possible options, this can be seen in the progression of Accuracy in the tests it does, that's why you should use 100 or more iterations (n_calls).